In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


Describe the bug
Got an email from JWW forwarded from folks in the Napa valley, who are looking at the Speed Maps. They reported not being able to see the following agencies

    Napa Valley Transportation Authority – Vine Transit
    Santa Rosa City Bus
    Sonoma County Transit
    County Connection
    WestCat

Can I get a quick reason why each of those agencies wasn't on the speedmap - for example - "No GTFS-RT, schedule information was insufficient in reason {x}, etc"


* Napa Valley Transportation Authority – Vine Transit

    * trip_ids in GTFS Schedule feed do not match trip_ids in GTFS-RT feed.

* Santa Rosa City Bus
    
    * Bad luck on choice of date, see below from GTFS Service Hours Issue Tracker
    * No data on 6/1/22 - the feed we got on that day only had Monday service defined? Seems like there was a temporary bad upload that they fixed on the 2nd (tracked in [sheet](https://docs.google.com/spreadsheets/d/16tcL3fPdYkrNajDNneSR1b5ImhFOCJ0gWFnEaXyg16A/edit#gid=0))
    * _fixable by choosing a different date, like 6/8_
    
    
* Sonoma County Transit

    * Apparent issue with GTFS-RT feed, valid endpoint but file returned appears either empty or not valid GTFS-RT.
    * No vehicle positions in warehouse, a quick look at Transit App shows no current RT either.
    
    
* County Connection

    * Airtable issue on our end, see above 
    * _fixable by entering data manually and rerunning, pending Airtable fix_
    

* WestCat

    * Apparent issue with GTFS-RT feed, valid endpoint but file returned appears either empty or not valid GTFS-RT.
    * No vehicle positions in warehouse, a quick look at Transit App shows no current RT either.



In [2]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    # >> filter(_.itp_id == itp_id)
    >> collect()
)

In [3]:
(airtable_organizations
 >> filter(-_.itp_id.isna())
 >> mutate(itp_id = _.itp_id.astype('int64'))
 # >> filter(_.caltrans_district == '04 - Oakland')
 >> filter(_.itp_id.isin([218, 301, 314, 61, 368]))

)

,itp_id,name,caltrans_district,website,ntp_id,drmt_organization_name
666,314,Sonoma County,04 - Oakland,None,['recCxVtrRwCEjfkNm'],None
696,301,City of Santa Rosa,04 - Oakland,None,['rectAf2e4ldkgt4v3'],['rec6rJ2ZovJDWCBZ6']
709,368,Western Contra Costa Transit Authority,04 - Oakland,None,['rec8FfQ984xt7gZcY'],None
716,218,Napa Valley Transportation Authority,04 - Oakland,None,['rec3GxaLefSC4zQYs'],['recHtk7XryNwJBqw8']


In [4]:
## county connection 61 not appearing in query?

In [5]:
analysis_date = dt.date(2022, 6, 1) ##wednesday, new tables

In [6]:
def vp_coverage(itp_id, analysis_date): 
    last_week = analysis_date - dt.timedelta(days = 7)
    next_week = analysis_date + dt.timedelta(days = 7)
    date_str = analysis_date.strftime('%Y-%m-%d')

    start = dt.datetime.combine(analysis_date, dt.time(0))
    end = start + dt.timedelta(days = 1, seconds = 2 * 60**2)

    df = query_sql(f"""
    SELECT DISTINCT itp_id AS calitp_itp_id, url_number AS calitp_url_number,
    dt AS date
    FROM `cal-itp-data-infra.external_gtfs_rt.vehicle_positions`
    WHERE itp_id = {itp_id} AND dt BETWEEN "{last_week}" AND "{next_week}"
    """)
    
    return df

# Sonoma County

* Apparent issue with GTFS-RT feed, valid endpoint but file returned appears either empty or not valid GTFS-RT.

In [7]:
# pbar = tqdm()

In [8]:
# agency = 314

In [9]:
# utils.get_trips(agency, analysis_date) >> head(5)

In [10]:
# sonoma_coverage = vp_coverage(agency, analysis_date)

In [11]:
# sonoma_coverage

In [12]:
# rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)

# City of Santa Rosa

* No data on 6/1/22 - the feed we got on that day only had Monday service defined? Seems like there was a temporary bad upload that they fixed on the 2nd (tracked in [sheet](https://docs.google.com/spreadsheets/d/16tcL3fPdYkrNajDNneSR1b5ImhFOCJ0gWFnEaXyg16A/edit#gid=0))

In [13]:
pbar = tqdm()

0it [00:00, ?it/s]

In [14]:
agency = 301

In [15]:
utils.get_trips(agency, analysis_date) >> head(5)

found parquet
cached parquet empty, will try a fresh query
getting trips...


,calitp_url_number,service_date,trip_key,trip_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,calitp_itp_id,route_id,route_short_name,route_long_name,route_desc,route_type


In [16]:
rt_day = rt.OperatorDayAnalysis(agency, dt.date(2022, 6, 8), pbar)

found parquet
found parquet
found parquet
found parquet


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


vehicle positions gdf must not be empty
less than 1km of data


In [17]:
rt_day.export_views_gcs()

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

In [18]:
import rt_filter_map_plot
rt_day = rt_filter_map_plot.from_gcs(agency, dt.date(2022, 6, 8))

found parquet


In [19]:
rt_day.set_filter(start_time='06:00', end_time='09:00')

In [20]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

/opt/conda/lib/python3.10/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py:281: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py:281: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [21]:
m

# WCCTA

* Apparent issue with GTFS-RT feed, valid endpoint but file returned appears either empty or not valid GTFS-RT.

In [22]:
pbar = tqdm()

0it [00:00, ?it/s]

In [23]:
agency = 368

In [24]:
# utils.get_trips(agency, analysis_date) >> head(5)

In [25]:
# wccta_coverage = vp_coverage(agency, analysis_date)

In [26]:
# wccta_coverage

In [27]:
# rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)

# Vine Transit

* trip_ids in GTFS Schedule feed do not match trip_ids in GTFS-RT feed.

In [28]:
pbar = tqdm()

0it [00:00, ?it/s]

In [29]:
agency = 218

In [30]:
# vine_coverage = vp_coverage(agency, analysis_date)

In [31]:
# vine_coverage

In [32]:
# vine_vp = utils.get_vehicle_positions(agency, analysis_date)

In [33]:
# vine_trips = utils.get_trips(agency, analysis_date)

In [34]:
# rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)

# County Connection

In [35]:
pbar = tqdm()

0it [00:00, ?it/s]

In [36]:
agency = 61

In [52]:
analysis_date = dt.date(2022, 6, 8)

In [53]:
rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)

getting trips...


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 60 seconds of data
less than 1km of data
less than 60 seconds of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data


In [56]:
rt_day.export_views_gcs()

AttributeError: 'RtFilterMapper' object has no attribute 'export_views_gcs'

In [55]:
import rt_filter_map_plot
rt_day = rt_filter_map_plot.from_gcs(agency, analysis_date)

found parquet


In [49]:
rt_day.set_filter(start_time='10:00', end_time='14:00')

In [50]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py:281: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py:281: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are lik

In [51]:
m